<font size=6><b>Lec08.NLP 언어모델
* NLP (Natural Language Processing, 자연어 처리)   * 언어 모델(Language Model, LM)은 언어라는 현상을 모델링하고자 단어 시퀀스(문장)에 확률을 할당(assign)하는 모델 


In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


sns.set()

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
# from IPython.core.display import display, HTML
from IPython.display import display, HTML

display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

import warnings
warnings.filterwarnings(action='ignore')

<font size=6><b>언어모델
* N-gram
* <b>빈도기반 
    * <b>BoW(Bag of Words)
    * <b>TF-IDF(Term Frequency - Inverse Document Frequency)
* 코사인 유사도(Cosine Similarity)

# N-gram
- 입력한 문자열을 N개의 기준 단위로 잘라서 분석하는 방법

In [2]:
text = 'Hello'
 
for i in range(len(text) - 1):             # 2-gram이므로 문자열의 끝에서 한 글자 앞까지만 반복함
    print(text[i], text[i + 1], sep='')    # 현재 문자와 그다음 문자 출력

He
el
ll
lo


In [3]:
text = 'this is python script'
words = text.split()                 # 공백을 기준으로 문자열을 분리하여 리스트로 만듦
 
for i in range(len(words) - 1):      # 2-gram이므로 리스트의 마지막에서 요소 한 개 앞까지만 반복함
    print(words[i], words[i + 1])    # 현재 문자열과 그다음 문자열 출력

this is
is python
python script


# BoW(Bag of Words)
* 희소행렬의 단점(백터의 수만큼 row가 만들어짐)을 극복하고자 등장  -> <font color='red'><b>단일 행렬(1차)</font></b>로 만듬 : 단어의 유무에 따라 [1,1,0,....] <b>+ 빈도</b>
* 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법

* dictionary items()

In [4]:
mydic= {"no":111, "name":"홍길동"}
print(  mydic.items() )
print(  mydic.keys() )
print(  mydic.values() )

dict_items([('no', 111), ('name', '홍길동')])
dict_keys(['no', 'name'])
dict_values([111, '홍길동'])


## nltk 불용어

In [5]:
from nltk.corpus import stopwords
stop_words_list = stopwords.words('english')
print('불용어 개수 :', len(stop_words_list))
print('불용어 10개 출력 :',stop_words_list[:10])

불용어 개수 : 179
불용어 10개 출력 : ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


## CountVectorizer() + 불용어

In [6]:
# doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."

from sklearn.feature_extraction.text import CountVectorizer

stop_words_list = stopwords.words('english')
my_word_list= ['because','I']

stop_words_list=stop_words_list+my_word_list

corpus = ['you know I want your love. because I love you.']
# vector = CountVectorizer(stop_words='english')  
# vector = CountVectorizer(stop_words=['because','I'])  
vector = CountVectorizer(stop_words=stop_words_list)  # 사용자 정의


# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector :', vector.fit_transform(corpus).toarray()) 

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary :',vector.vocabulary_)

vocab_sorted=sorted(vector.vocabulary_.items(),key=lambda x:x[1],reverse=False) # dict sorting   # reverse=False : ascending
print(vocab_sorted)

bag of words vector : [[1 2 1]]
vocabulary : {'know': 0, 'want': 2, 'love': 1}
[('know', 0), ('love', 1), ('want', 2)]


# DTM
문서 단어 행렬(DTM, Document-Term Matrix)
* <b>다수의 BoW : 각 문서에 대한 BoW를 하나의 행렬로 만든 것</b>
* 각 문서에서 등장한 단어의 빈도를 행렬의 값으로 표기해 서로 다른 문서들을 비교


* <단점>
    - <font color='red'><b>희소 표현(Sparse representation)</b> : 단어 집합의 크기 == 벡터의 차원이 되고 대부분의 값이 0이 된다
    - <font color='red'><b>단순 빈도 수 기반 접근</b> : '이다'와 같은 중요하지 않은 최빈도 단어로 문서를 연관지으면??
    - <font color='blue'><b> TF-IDF 필요 (DTM에 불용어와 중요한 단어에 대해서 가중치)</b></font>

## 실습

In [44]:
corpus_list = ['먹고 싶은 사과',
        '먹고 싶은 바나나',
        '길고 노란 바나나 바나나',
        '저는 과일이 좋아요']

# --- BoW : 사전에 있으면 1 없으면 0 또있으면 누적 [1,0,0,1,2]
vector = CountVectorizer()  

bow = vector.fit_transform(corpus_list).toarray()
print(vector.fit_transform(corpus_list))
print('BoW : \n', bow)   #COO(밀집) CSR(희소)

# --- BoW : 에 있는 단어가 숫자 ?로 변경되었는지 확인
print('vocabulary \n:',vector.vocabulary_)
# --- BoW : 에 있는 단어를 index 순으로 정렬
vocab_sorted = sorted(vector.vocabulary_.items(), key = lambda x:x[1], reverse = False)
print(vocab_sorted)


# --- BoW : 에 있는 단어를 index 순으로 정렬 --> 중에서 key(단어)만 가져오기
vocab_word = [t[0] for t in vocab_sorted ]
# print(vocab_word)

vocab_word = dict(vocab_sorted).keys()
print(vocab_word)
    

# --- DTM : BoW집합  (인덱스:문서번호, 컬럼:단어들(vocab_word),  값:BoW)
df = pd.DataFrame(bow, columns=vocab_word)
df

  (0, 3)	1
  (0, 6)	1
  (0, 5)	1
  (1, 3)	1
  (1, 6)	1
  (1, 4)	1
  (2, 4)	2
  (2, 1)	1
  (2, 2)	1
  (3, 7)	1
  (3, 0)	1
  (3, 8)	1
BoW : 
 [[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
vocabulary 
: {'먹고': 3, '싶은': 6, '사과': 5, '바나나': 4, '길고': 1, '노란': 2, '저는': 7, '과일이': 0, '좋아요': 8}
[('과일이', 0), ('길고', 1), ('노란', 2), ('먹고', 3), ('바나나', 4), ('사과', 5), ('싶은', 6), ('저는', 7), ('좋아요', 8)]
dict_keys(['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요'])


,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


- 내꺼

In [8]:
corpus = ['먹고 싶은 사과',
'먹고 싶은 바나나',
'길고 노란 바나나 바나나',
'저는 과일이 좋아요']

vector = CountVectorizer()
print(vector.fit_transform(corpus))             # COO(밀집)
print(vector.fit_transform(corpus).toarray())   # CSR(희소)
print(vector.vocabulary_)

  (0, 3)	1
  (0, 6)	1
  (0, 5)	1
  (1, 3)	1
  (1, 6)	1
  (1, 4)	1
  (2, 4)	2
  (2, 1)	1
  (2, 2)	1
  (3, 7)	1
  (3, 0)	1
  (3, 8)	1
[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
{'먹고': 3, '싶은': 6, '사과': 5, '바나나': 4, '길고': 1, '노란': 2, '저는': 7, '과일이': 0, '좋아요': 8}


In [9]:
vocab_sorted=sorted(vector.vocabulary_.items(),key=lambda x:x[1],reverse=False) # dict sorting   # reverse=False : ascending
print(vocab_sorted)
vocab_word=dict(vocab_sorted).keys()
print(vocab_word)

[('과일이', 0), ('길고', 1), ('노란', 2), ('먹고', 3), ('바나나', 4), ('사과', 5), ('싶은', 6), ('저는', 7), ('좋아요', 8)]
dict_keys(['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요'])


In [10]:
corpus_DTM=pd.DataFrame(vector.fit_transform(corpus).toarray(),columns=vocab_word,index=['문장1','문장2','문장3','문장4'])
corpus_DTM

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
문장1,0,0,0,1,0,1,1,0,0
문장2,0,0,0,1,1,0,1,0,0
문장3,0,1,1,0,2,0,0,0,0
문장4,1,0,0,0,0,0,0,1,1


# TF_IDF
* 단어 빈도-역 문서 빈도(TF-IDF, Term Frequency-Inverse Document Frequency)
* <b>우선 DTM을 만든 후, TF-IDF 가중치를 부여</b>
* 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 <b>특정 단어의 중요도</b>를 구하는 작업 등에 활용
* TF-IDF = TF * IDF

* 문서(d), 단어(t), 문서총개수(n)
* <font color='red'> $tf(d,t)$ : 특정 문서 d에서의 특정 단어 t의 등장 횟수</font>   
    - ex. 바나나는 문장2에서 1번 등장했다(가로 확인)<br><br>
* <font color='red'>  $df(t)$ : 특정 단어 t가 등장한 문서의 수 </font>    
    - ex. 바나나는 2개의 문서에서 발견되었다(세로 확인)<br><br>
* <font color='red'>  $idf(d, t) = log (\frac{n}{1+df(t)})$ : df(t) 역수</font>

## [실습]

In [11]:
corpus_list = ['먹고 싶은 사과',
'먹고 싶은 바나나',
'길고 노란 바나나 바나나',
'저는 과일이 좋아요']

# 상위코드 참조 : vocab
vocab=['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

## 공통함수

In [12]:
def tf(t, d): #DTM : BoW값 계산
    return d.count(t)

def idf(t):
    # -- 문서들(docs)에서 해당 단어(t)의 빈도수
    df = 0
    for doc in corpus_list:
        df += t in doc
    # -- idf = df의 역수(1 / df)
    return np.log(N/(df + 1))  #  log1p(N/(df))    # N : 문서개수

def tfidf(t, d):
    return tf(t,d)* idf(t)

### TF(DTM = BoWs)
- 특정 문서 d에서의 특정 단어 t의 등장 횟수
- 한 문장에서 바나나 몇번 나와?

In [13]:
N = len(corpus_list)               #문서의갯수 : 4
wcnt_list = []
for d in corpus_list :   # d : 문서
    for t in vocab :       # vocab size : 9
        wcnt_list.append( tf(t, d) )  # d문서에 t단어가 출현한 빈도수
wcnt_list = np.array(wcnt_list).reshape(N,-1)
wcnt_list

array([[0, 0, 0, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 1]])

### IDF
* "특정 단어 t가 등장한 문서의 수" 의 역수
* "바나나가 나오는 문서 갯수"  의 역수

In [14]:
dcnt_list=[]
for i in vocab :
    dcnt_list.append(idf(t))
dcnt_list

[0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453,
 0.6931471805599453]

### TF-IDF

In [15]:
tf_arr= wcnt_list
idf_arr=np.array(dcnt_list)
tfidf_arr=tf_arr*idf_arr
tfidf_arr

array([[0.        , 0.        , 0.        , 0.69314718, 0.        ,
        0.69314718, 0.69314718, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.69314718, 0.69314718,
        0.        , 0.69314718, 0.        , 0.        ],
       [0.        , 0.69314718, 0.69314718, 0.        , 1.38629436,
        0.        , 0.        , 0.        , 0.        ],
       [0.69314718, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.69314718, 0.69314718]])

In [16]:
N=len(corpus)  # 문서의 개수 : 4
TF_IDF_list=[]
for d in corpus_list :   # d : 문서
    for t in vocab :  
        TF_IDF_list.append(tfidf(t, d))
TF_IDF_list = np.array(TF_IDF_list).reshape(N,-1)
TF_IDF_list

array([[0.        , 0.        , 0.        , 0.28768207, 0.        ,
        0.69314718, 0.28768207, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.28768207, 0.28768207,
        0.        , 0.28768207, 0.        , 0.        ],
       [0.        , 0.69314718, 0.69314718, 0.        , 0.57536414,
        0.        , 0.        , 0.        , 0.        ],
       [0.69314718, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.69314718, 0.69314718]])

In [17]:
TF_IDF_df=pd.DataFrame(TF_IDF_list,columns=vocab)
TF_IDF_df # 값이 클수록 중요도 높음

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


<pre>
<font size=3 color=blue><b>
obj     = CountVectorizer()
dtm_arr = obj.fit_transform(corpus_list).toarray()
vocab   = obj.vocabulary_

obj       = TfidfVectorizer()
tfidf_arr = obj.fit_transform(corpus_list).toarray()
vocab     = obj.vocabulary_


### <font color=red size=5><b>sklearn.CountVectorizer() : DTM(BoWs) - freq단어
* from sklearn.feature_extraction.text import CountVectorizer

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

corpus_list = ['먹고 싶은 사과',
'먹고 싶은 바나나',
'길고 노란 바나나 바나나',
'저는 과일이 좋아요']

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
bows=vector.fit_transform(corpus_list).toarray()
print(bows)

# 각 단어와 맵핑된 인덱스 출력
vocab=vector.vocabulary_
print(vocab)

bows_df=pd.DataFrame(bows,columns=vocab.keys())
bows_df 

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
{'먹고': 3, '싶은': 6, '사과': 5, '바나나': 4, '길고': 1, '노란': 2, '저는': 7, '과일이': 0, '좋아요': 8}


,먹고,싶은,사과,바나나,길고,노란,저는,과일이,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


### <font color=red size=5><b>sklearn. TfidfVectorizer() : DTM(단어freq)-doc 중요도
* from sklearn.feature_extraction.text import TfidfVectorizer
* (IDF의 로그항의 분자에 1을 더해주며, 로그항에 1을 더해주고, TF-IDF에 L2 정규화라는 방법으로 값을 조정

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus_list = ['먹고 싶은 사과',
'먹고 싶은 바나나',
'길고 노란 바나나 바나나',
'저는 과일이 좋아요']


tfidfv = TfidfVectorizer()
bows=tfidfv.fit_transform(corpus_list).toarray()
print(bows)

vocab=tfidfv.vocabulary_
print(vocab)

tfidf_df=pd.DataFrame(bows,columns=vocab.keys())
tfidf_df 

[[0.         0.         0.         0.52640543 0.         0.66767854
  0.52640543 0.         0.        ]
 [0.         0.         0.         0.57735027 0.57735027 0.
  0.57735027 0.         0.        ]
 [0.         0.47212003 0.47212003 0.         0.7444497  0.
  0.         0.         0.        ]
 [0.57735027 0.         0.         0.         0.         0.
  0.         0.57735027 0.57735027]]
{'먹고': 3, '싶은': 6, '사과': 5, '바나나': 4, '길고': 1, '노란': 2, '저는': 7, '과일이': 0, '좋아요': 8}


,먹고,싶은,사과,바나나,길고,노란,저는,과일이,좋아요
0,0.00000,0.00000,0.00000,0.526405,0.00000,0.667679,0.526405,0.00000,0.00000
1,0.00000,0.00000,0.00000,0.577350,0.57735,0.000000,0.577350,0.00000,0.00000
2,0.00000,0.47212,0.47212,0.000000,0.74445,0.000000,0.000000,0.00000,0.00000
3,0.57735,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.57735,0.57735


# <font color=red size=5><b>코사인 유사도(Cosine Similarity)
* 벡터의 크기가 아니라 방향(패턴)에 초점
* <b>두 문서(문장) 간의 유사한 관계 정도를 나타냄</b>
* ref : https://wikidocs.net/24603

In [20]:
tf_arr    = wcnt_list
idf_arr   = np.array(dcnt_list)
tfidf_arr = tf_arr * idf_arr
tfidf_arr

array([[0.        , 0.        , 0.        , 0.69314718, 0.        ,
        0.69314718, 0.69314718, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.69314718, 0.69314718,
        0.        , 0.69314718, 0.        , 0.        ],
       [0.        , 0.69314718, 0.69314718, 0.        , 1.38629436,
        0.        , 0.        , 0.        , 0.        ],
       [0.69314718, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.69314718, 0.69314718]])

<img width=500 src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbpLIxd%2Fbtq03F9AfHS%2FRTzzEkvHWvqQ5XfQh6rkY1%2Fimg.png"><br>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb9gdZi%2FbtqGU2DGM9W%2FRM2ZgrpVZ4356nKEUBM8U0%2Fimg.png" width=400><br>
* Cosine 0° : 두 벡터(단어)간에 매우 유사한 관계를 지닌다. Cosine Similarity값은 1이 된다.
* Cosine 90° : 두 벡터간에 관계가 없음을 의미한다. Cosine Similarity값은 0이 된다. 
* Cosine 180° : 두 벡터간에 반대관계를 의미한다. Cosine Similarity값은 -1이 된다.
* <b>문서 간 유사도  : 0 ~ 1</b>

<img width=300  src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbayZ5E%2Fbtq04KWSnlM%2FAkYrOmeom4jOCLv2fBf1HK%2Fimg.png"><br>

## 공통함수

In [21]:
import numpy as np
from numpy.linalg import norm  #--정규화 ||A||
#from numpy import dot

def my_cos_sim(A,B):
    cos_sim = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
    print(cos_sim)   
    
# sklearn...vectorize.. [1,2,3,]
doc1 = [0,1,2,1]
doc2 = [1,0,1,1]
doc3 = [2,0,2,2]
my_cos_sim(doc1,doc2)
my_cos_sim(doc1,doc3)
my_cos_sim(doc2,doc3)    

0.7071067811865476
0.7071067811865476
1.0000000000000002


In [22]:
A = df.loc[0].values
B = df.loc[1].values
print(A, B)
res1 = np.dot(A, B)
res2 = np.sqrt(np.sum(A**2)) * np.sqrt(np.sum(B**2))
cs = res1 / res2
print(f"코사인유사도 : {cs}")

# np.sqrt(np.sum(B**2)) 계산 원리 -------------
# B_sum = 0
# for b in B:
#     B_sum +=  b**2
# np.sqrt(B_sum)

[0 0 0 1 0 1 1 0 0] [0 0 0 1 1 0 1 0 0]
코사인유사도 : 0.6666666666666667


In [23]:
def my_cos_sim(A,B):  
    cos_sim = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))   # norm : 제곱 합의 루트
    print(cos_sim)   
my_cos_sim(A,B)

0.6666666666666667


## 사용예

### Tokenizer

In [24]:
### Tokenizer()
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

text_list = ["어제 비가 와서 날씨가 추워졌다.",
             "나는 고양이를 좋아한다.",
             "나는 강아지를 좋아한다."]

token = Tokenizer()
token.fit_on_texts(text_list)

text_seq_list = token.texts_to_sequences(text_list)
print(text_seq_list)
vocab = token.word_index
print(vocab)


max_length = max(len(sent.split()) for sent in text_list )
print('최대길이(Voca Size):', max_length)


padding_res = pad_sequences(sequences=text_seq_list,   #[[ 1, 2, ..] , ['1, 2, ..]]
                   maxlen=5,      #voca_size :문장들이 가지고 있는 단어갯수 중 최대 단어수
                   padding="pre"  #앞에서부터 0을 채워서 제일 긴 길이에 맞추기 pre/post
                  )
print(padding_res)


cs1 = my_cos_sim(padding_res[0], padding_res[1])
cs2 = my_cos_sim(padding_res[0], padding_res[2])
cs3 = my_cos_sim(padding_res[1], padding_res[2])
print(cs1, cs2, cs3)

[[3, 4, 5, 6, 7], [1, 8, 2], [1, 9, 2]]
{'나는': 1, '좋아한다': 2, '어제': 3, '비가': 4, '와서': 5, '날씨가': 6, '추워졌다': 7, '고양이를': 8, '강아지를': 9}
최대길이(Voca Size): 5
[[3 4 5 6 7]
 [0 0 1 8 2]
 [0 0 1 9 2]]
0.6941980239122336
0.6774960231074738
0.9995786102379159
None None None


### sklearn.CountVectorizer() : DTM(BoWs) - freq단어
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

text_list = ["어제 비가 와서 날씨가 추워졌다.",
             "나는 고양이를 좋아한다.",
             "나는 강아지를 좋아한다."]

vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
bows=vector.fit_transform(text_list).toarray()
print(bows)

# 각 단어와 맵핑된 인덱스 출력
vocab=vector.vocabulary_
print(vocab)

bows_df=pd.DataFrame(bows,columns=vocab.keys())
bows_df 

[[0 0 0 1 1 1 1 0 1]
 [0 1 1 0 0 0 0 1 0]
 [1 0 1 0 0 0 0 1 0]]
{'어제': 5, '비가': 4, '와서': 6, '날씨가': 3, '추워졌다': 8, '나는': 2, '고양이를': 1, '좋아한다': 7, '강아지를': 0}


,어제,비가,와서,날씨가,추워졌다,나는,고양이를,좋아한다,강아지를
0,0,0,0,1,1,1,1,0,1
1,0,1,1,0,0,0,0,1,0
2,1,0,1,0,0,0,0,1,0


In [26]:
A=bows_df.iloc[0].values
B=bows_df.iloc[1].values
C=bows_df.iloc[2].values

In [27]:
my_cos_sim(A,B)
my_cos_sim(B,C)
my_cos_sim(A,C)

0.0
0.6666666666666667
0.0


### sklearn. TfidfVectorizer() : DTM(단어freq)-doc 중요도
* from sklearn.feature_extraction.text import TfidfVectorizer 

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer()
bows=tfidfv.fit_transform(text_list).toarray()
print(bows)

vocab=tfidfv.vocabulary_
print(vocab)

tfidf_df=pd.DataFrame(bows,columns=vocab.keys())
tfidf_df 

[[0.         0.         0.         0.4472136  0.4472136  0.4472136
  0.4472136  0.         0.4472136 ]
 [0.         0.68091856 0.51785612 0.         0.         0.
  0.         0.51785612 0.        ]
 [0.68091856 0.         0.51785612 0.         0.         0.
  0.         0.51785612 0.        ]]
{'어제': 5, '비가': 4, '와서': 6, '날씨가': 3, '추워졌다': 8, '나는': 2, '고양이를': 1, '좋아한다': 7, '강아지를': 0}


,어제,비가,와서,날씨가,추워졌다,나는,고양이를,좋아한다,강아지를
0,0.000000,0.000000,0.000000,0.447214,0.447214,0.447214,0.447214,0.000000,0.447214
1,0.000000,0.680919,0.517856,0.000000,0.000000,0.000000,0.000000,0.517856,0.000000
2,0.680919,0.000000,0.517856,0.000000,0.000000,0.000000,0.000000,0.517856,0.000000


In [29]:
A=tfidf_df.iloc[0].values
B=tfidf_df.iloc[1].values
C=tfidf_df.iloc[2].values

In [30]:
my_cos_sim(A,B)
my_cos_sim(B,C)
my_cos_sim(A,C)

0.0
0.5363499141045837
0.0


In [31]:
for i in range(len(tfidf_df)):
    for j in range(len(tfidf_df)):
        if i<j:
            A=tfidf_df.iloc[i]
            B=tfidf_df.iloc[j]
            my_cos_sim(A,B)

0.0
0.0
0.5363499141045837


# [실습] 영화 추천 시스템
- ref : https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset

In [32]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

## Data Load

In [33]:
data=pd.read_csv("./datasets/movies_metadata.csv")[:20000]
data.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  20000 non-null  object 
 1   belongs_to_collection  2399 non-null   object 
 2   budget                 20000 non-null  object 
 3   genres                 20000 non-null  object 
 4   homepage               3055 non-null   object 
 5   id                     20000 non-null  object 
 6   imdb_id                19993 non-null  object 
 7   original_language      19999 non-null  object 
 8   original_title         20000 non-null  object 
 9   overview               19865 non-null  object 
 10  popularity             19998 non-null  object 
 11  poster_path            19907 non-null  object 
 12  production_companies   19999 non-null  object 
 13  production_countries   19999 non-null  object 
 14  release_date           19983 non-null  object 
 15  re

## 결측처리

In [35]:
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())
data['overview'] = data['overview'].fillna('')
print('overview 열의 결측값의 수:',data['overview'].isnull().sum())

overview 열의 결측값의 수: 135
overview 열의 결측값의 수: 0


In [36]:
# stop_words_list = stopwords.words('english')
# print('불용어 개수 :', len(stop_words_list))
# print('불용어 10개 출력 :',stop_words_list[:10])

## TF-IDF를 이용해 코사인 유사도 계산
* cosine_sim

In [37]:
tfidf = TfidfVectorizer(stop_words='english') # 중요도 측정
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

print(tfidf_matrix[0:5].toarray())
# print(tfidf.vocabulary_)

TF-IDF 행렬의 크기(shape) : (20000, 47487)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [38]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix) # 문서간 유사도 측정
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [39]:
# overview의 유사도
cosine_df=pd.DataFrame(cosine_sim,columns=data['title'],index=data['title'])
cosine_df.head()

title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,The American President,Dracula: Dead and Loving It,Balto,Nixon,Cutthroat Island,Casino,Sense and Sensibility,Four Rooms,Ace Ventura: When Nature Calls,Money Train,Get Shorty,Copycat,Assassins,Powder,Leaving Las Vegas,Othello,Now and Then,Persuasion,The City of Lost Children,Shanghai Triad,Dangerous Minds,Twelve Monkeys,Wings of Courage,Babe,Carrington,Dead Man Walking,Across the Sea of Time,It Takes Two,Clueless,"Cry, the Beloved Country",Richard III,Dead Presidents,Restoration,Mortal Kombat,To Die For,How To Make An American Quilt,Se7en,Pocahontas,When Night Is Falling,The Usual Suspects,...,Kiss and Tell,"Oui, mais...",Space Adventure Cobra,L'aventure c'est l'aventure,A Season for Miracles,The Awful Truth,1911,Looking for Jackie,The Shaolin Temple,Shaolin Temple 2: Kids from Shaolin,Carol Channing: Larger Than Life,Memory,Martial Arts of Shaolin,Snowmageddon,Stand Up and Fight,Jazz,Barbara,A Liar's Autobiography: The Untrue Story of Monty Python's Graham Chapman,Little Miss Broadway,Miss Annie Rooney,Mr. Belvedere Goes to College,The Hobbit: An Unexpected Journey,Mon Paradis - Der Winterpalast,No Flesh Shall Be Spared,I am Von Höfler Variation on Werther,Born to Defend,Dragon Fight,The Master,Kung Fu Cult Master,The Fitzgerald Family Christmas,Hyde Park on Hudson,Lay the Favorite,Audition/Talent Competition,All's Faire in Love,Crooked Arrows,The Enforcer,Dr. Wai in the Scriptures with No Words,Once Upon a Time in China and America,Contract Killer,The Executor,Never Say... Never!,Calmos,How to Make Love to a Woman,"After Fall, Winter",Violeta Went to Heaven,Rebellion,Versailles,Two in the Wave,Lotte Reiniger: Homage to the Inventor of the Silhouette Film,"RKO Production 601: The Making of 'Kong, the Eighth Wonder of the World'"
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.015757,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.041140,0.000000,0.0,0.009941,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.019831,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.006348,0.000000,0.0,0.009321,0.00000,0.000000,0.000000,0.013822,...,0.000000,0.0,0.011365,0.0,0.000000,0.0,0.011791,0.014688,0.013539,0.007355,0.011161,0.000000,0.01452,0.000000,0.000000,0.0,0.000000,0.000000,0.019853,0.027106,0.00000,0.000000,0.0,0.0,0.000000,0.00000,0.117982,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.009979,0.060997,0.000000,0.0,0.0,0.024585,0.0,0.0,0.0
Jumanji,0.015757,1.000000,0.049073,0.0,0.000000,0.051830,0.00000,0.000000,0.106355,0.0,0.007616,0.0,0.0,0.008233,0.0,0.0,0.000000,0.028753,0.000000,0.0,0.000000,0.0,0.006043,0.000000,0.0,0.024285,0.024941,0.006489,0.0,0.000000,0.000000,0.047250,0.0,0.005560,0.004326,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.025758,0.0,0.000000,0.01848,0.008127,0.000000,0.016429,...,0.000000,0.0,0.004063,0.0,0.000000,0.0,0.006281,0.000000,0.000000,0.000000,0.030391,0.000000,0.00000,0.027595,0.043131,0.0,0.011296,0.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0.007418,0.01348,0.009529,0.000000,0.0,0.060063,0.0,0.0,0.000000,0.013191,0.000000,0.000000,0.0,0.00000,0.0,0.0,0.029242,0.000000,0.000000,0.004271,0.0,0.0,0.000000,0.0,0.0,0.0
Grumpier Old Men,0.000000,0.049073,1.000000,0.0,0.025005,0.000000,0.00000,0.006500,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.018349,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.011910,0.0,0.010392,0.000000,0.006408,0.0,0.006683,0.000000,0.000000,0.005282,0.0,0.0,0.0,0.0,0.000000,0.000000,0.007480,0.0,0.000000,0.00000,0.016500,0.016821,0.000000,...,0.000000,0.0,0.000000,0.0,0.007415,0.0,0.000000,0.000000,0.000000,0.015443,0.000000,0.007561,0.00000,0.031215,0.000000,0.0,0.

## 유사영화 Top10 검색
* 영화제목을 넣으면 코사인유사도 Top10 영화 제목을 리턴한다.

In [40]:
cosine_df.loc['The Dark Knight Rises'].sort_values(ascending=False)[1:11]

title
The Dark Knight                            0.321521
Batman Forever                             0.315328
Batman Returns                             0.305010
Batman: Under the Red Hood                 0.296967
Batman                                     0.276030
Batman Beyond: Return of the Joker         0.236289
Batman: Year One                           0.209219
Batman: The Dark Knight Returns, Part 1    0.206999
Batman: Mask of the Phantasm               0.196292
Batman Begins                              0.179930
Name: The Dark Knight Rises, dtype: float64

## 비추
* 교재 예제 코드 참고용

### 영화검색을 위한 색인 dict 만들기
* title_to_index : 제목 넣으면 영화 인덱스 가져오기
* movie_dic = {'영화제목' : 영화의인덱스 }

In [41]:
# movie_dic = {'영화제목' :영화인덱스}
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)
print(data.loc[idx,'title'])

4
Father of the Bride Part II


In [42]:
# -------------------------------------------------------------------
# 영화의 제목을 입력 시 코사인 유사도 top10 영화를 찾아내는 함수
# -------------------------------------------------------------------
def my_top10_movie(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]  # 본인 제외 나머지 overview 가져오기
    print(sim_scores)

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [tupl[0] for tupl in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data.loc[ movie_indices,'title']

In [43]:
my_top10_movie('The Dark Knight Rises')

[(12481, 0.32152142350025487), (150, 0.31532758128008986), (1328, 0.30500953745851006), (15511, 0.29696687233514857), (585, 0.2760295597760307), (9230, 0.2362886826517737), (18035, 0.20921858266951263), (19792, 0.2069989680094894), (3095, 0.1962924866191582), (10122, 0.17992977744685185)]


12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object